In [1]:
%load_ext autoreload
%autoreload 2

from jenga.tasks.openml import OpenMLTask
from jenga.cleaning.cleaners import AutoGluonCleaner
from jenga.corruptions.numerical import Scaling
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.autocleaning import AutoCleaningEvaluator
import numpy as np

/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [2]:
task = OpenMLTask(0)

cleaner = AutoGluonCleaner(numeric_columns=task.numerical_columns, 
                           categorical_columns=task.categorical_columns)

In [4]:
model = task.fit_baseline_model(task.train_data, task.train_labels)
cleaner.fit(task.train_data)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.3s finished


Found 15 predictable columns: ['c', 'l', 'o', 'r', 'z', 'a', 'b', 'a', 'f', 'd', 'j', 'o', 't', 'a', 'c']


In [5]:
evaluator = AutoCleaningEvaluator(task, cleaner)

In [9]:
corruptions = []
for impacted_column in task.numerical_columns + task.categorical_columns:
    for fraction in [0.99, 0.5, 0.25, 0.1, 0.01]:
        if impacted_column in task.numerical_columns:
            corruptions.append(Scaling(impacted_column, fraction))
        for missingness in ['MCAR', 'MAR', 'MNAR']:
            corruption = MissingValues(impacted_column, fraction, missingness=missingness, na_value=np.nan)
            corruptions.append(corruption)

In [10]:
evaluator.evaluate(model, 5, *corruptions[:2])

NameError: name 'y_test' is not defined

In [8]:
debug

> /Users/felix/code/jenga/jenga/evaluation/autocleaning.py(51)evaluate()
     49                 test_data_copy = self.__task.test_data.copy(deep=True)
     50 
---> 51                 corrupted_data = corruption.transform(test_data_copy)
     52                 corrupted_scores += self.compute_eval_score(model, corrupted_data)
     53 

ipdb> corruption
<class 'jenga.corruptions.numerical.Scaling'>
ipdb> q
